In [ ]:
import numpy as np
import re
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import data

!wget -O a.txt "https://drive.google.com/uc?export=download&id=1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56"

a = np.loadtxt('a.txt')
a_t = torch.tensor(a)
a_t = a_t.type(torch.LongTensor)

y_real = []
y_imag = []
#with open('drive/MyDrive/y.txt') as input_file:
with open('y.txt') as input_file:
    for line in input_file:
      temp = re.sub(r'[()]', "", line)
      temp = complex(temp)
      y_real.append(temp.real)
      y_imag.append(temp.imag)

--2023-01-25 10:33:04--  https://drive.google.com/uc?export=download&id=1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56
Resolving drive.google.com (drive.google.com)... 74.125.69.139, 74.125.69.101, 74.125.69.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.69.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7i3i3a37kdrcp8ndukccfg947gnnt0oe/1674642750000/00806258749903615801/*/1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56?e=download&uuid=96cb40a4-baa3-4114-b0d4-0da1e6d2e0ce [following]
--2023-01-25 10:33:06--  https://doc-14-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7i3i3a37kdrcp8ndukccfg947gnnt0oe/1674642750000/00806258749903615801/*/1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56?e=download&uuid=96cb40a4-baa3-4114-b0d4-0da1e6d2e0ce
Resolving doc-14-80-docs.googleusercontent.com (doc-14-80-docs.googleusercontent.com)... 74.125.132.132, 2607:f8b0

In [ ]:
# Conversion to TensorFlow

y_real_t = torch.Tensor(y_real).reshape(-1,1)
y_imag_t = torch.Tensor(y_imag).reshape(-1,1)
Y = torch.stack((y_real_t, y_imag_t), -1)

In [ ]:
# Divide your data into training, validation and test dataset

a_train, a_test =  train_test_split(a_t, test_size=0.2, shuffle=False)
y_train, y_test =  train_test_split(Y, test_size=0.2, shuffle=False)

a_val, a_test =  train_test_split(a_t, test_size=0.5, shuffle=False)
y_val, y_test =  train_test_split(Y, test_size=0.5, shuffle=False)

In [ ]:
# Define nonlinear NN for equalizer

class NonlinEq(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(2, 50)
        self.relu1 = nn.ReLU()
        self.lin2 = nn.Linear(50, 20)
        self.relu2 = nn.ReLU()
        self.lin3 = nn.Linear(20, 2)

    def forward(self, y):
        y = self.relu1(self.lin1(y))
        y = self.relu2(self.lin2(y))
        return self.lin3(y)[::2]

In [ ]:
# Define linear NN for Demapping

class Demapper(nn.Module):
    def __init__(self):
        super().__init__()
        self.out = nn.Linear(2, 16)

    def forward(self, y):
        return self.out(y)

In [ ]:
# Define loss function

loss_fn = nn.CrossEntropyLoss()
lr = 0.05  # learning rate
num_epochs = 151

In [ ]:
# Initialize NN

torch.manual_seed(5)  
nleq = NonlinEq()
demap = Demapper()
optimizer = optim.Adam(list(nleq.parameters()) + list(demap.parameters()), lr=lr)

loss_last_batch = []
loss_val_set = []

In [ ]:
# Trainings loop

for j in range(num_epochs):
    y_eq = nleq(Y)
    logit = demap(y_eq).reshape(-1, 16)
    loss = loss_fn(logit, a_t)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # store loss of the last batch 
    loss_last_batch.append(loss.detach().numpy())
    # store loss of validation set 
    loss_val = loss_fn(demap(nleq(y_val)).reshape(-1, 16), a_val)
    loss_val_set.append(loss_val.detach().numpy())

    if j % 25 == 0:
        print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')

epoch 0: Loss = 2.8066
epoch 25: Loss = 0.6975
epoch 50: Loss = 0.3503
epoch 75: Loss = 0.2189
epoch 100: Loss = 0.1556
epoch 125: Loss = 0.1241
epoch 150: Loss = 0.1061


In [ ]:
# Save result of the NN

torch.save(loss_last_batch, 'last_batch1.pt')
torch.save(loss_val_set, 'val1.pt')
torch.save(nleq, 'equal1.pth')
torch.save(demap, 'demap.pth')